# Installation and Setup 🛠️

Before diving into fine‑tuning, we need to install **Unsloth**, CUDA toolchain, and other dependencies. The following cell installs everything you need so you can load the model and prepare the data.

In [1]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!mamba create -y -p "$HOME/cuda128" -c nvidia/label/cuda-12.8.1 --no-default-packages \
    cuda-nvcc=12.8.61 cuda-cudart-dev=12.8.90

import os, shutil, subprocess

CUDA_HOME = os.path.expanduser("~/cuda128")  # or "/tmp/cuda128"
os.environ["CUDA_HOME"] = CUDA_HOME
os.environ["PATH"] = f"{CUDA_HOME}/bin:" + os.environ.get("PATH","")
os.environ["LD_LIBRARY_PATH"] = os.environ.get("LD_LIBRARY_PATH","") + f":{CUDA_HOME}/lib64"

print("nvcc:", shutil.which("nvcc"))
print(subprocess.check_output(["nvcc","-V"], text=True))

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-r0v30h82/unsloth_c9808e8254b84c12b6b30be7c7832d40
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-r0v30h82/unsloth_c9808e8254b84c12b6b30be7c7832d40
  Resolved https://github.com/unslothai/unsloth.git to commit 8c155a2c07fc6521c869e44fb5f43b93fc0b81ff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

Looking for: ['cuda-nvcc=12.8.61', 'cuda-cudart-dev=12.8.90']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
[+] 0.1s
nvidia/label/cuda-12.8.1/linux-64 ━━━━━━━━╸━━━━━━━━━━━   0.0 B @  ??.?MB/s  0.1s
nvidia/label/cuda-12.8.1/noarch   ━━━━╸━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s  0.1snvidia/label/cuda-12.8.1/linux-64                             No change
[+] 0.2s
nvidia/lab

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.045 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.10.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Data Preparation 📊

We will load the Titanic dataset from Hugging Face and prepare it for fine‑tuning. After splitting into training and test sets, we convert the features into a conversational format that the model can understand.

In [4]:
# Load the Titanic dataset and split into training and test sets
from datasets import load_dataset
dataset = load_dataset(
    "csv",
    data_files="https://huggingface.co/datasets/unsloth/datasets/raw/main/titanic.csv",
    split="train",
)

# Display dataset columns and a sample row
print(dataset.column_names)
print(dataset[0])

# Split into train and test (80‑20 split)
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
{'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': None, 'Embarked': 'S'}
Train size: 712, Test size: 179


# Convert Dataset to Conversational Format 🗣️

Next, we convert each row of the training dataset into a conversational prompt/response format. This makes it easier for the model to learn by reading natural language descriptions of passenger attributes and predicting whether they survived.

In [5]:
# from unsloth import to_sharegpt
# dataset_simple = to_sharegpt(
#     dataset,
#     merged_prompt = "[[Their age is {Age}.\n]][[They paid ${Fare} for the trip.\n]]",
#     output_column_name = "Survived",
# )

In [7]:
# Convert the training dataset into the ShareGPT-style format and inspect a sample
from unsloth import to_sharegpt, standardize_sharegpt
from pprint import pprint

train_dataset = to_sharegpt(
    train_dataset,
    merged_prompt = 
        "[[The passenger embarked from {Embarked}.]]"
        "[[They are {Sex}.]]"
        "[[They have {Parch} parents and childen.]]"
        "[[They have {SibSp} siblings and spouses.]]"
        "[[Their passenger class is {Pclass}.]]"
        "[[Their age is {Age}.]]"
        "[[They paid ${Fare} for the trip.]]",
    conversation_extension = 5,
    output_column_name = "Survived",
)

# Inspect the first formatted training example
pprint(train_dataset[0])

# Standardize the ShareGPT format
train_dataset = standardize_sharegpt(train_dataset)

# View the standardized example
train_dataset[0]

{'conversations': [{'from': 'human',
                    'value': 'The passenger embarked from S.\n'
                             'They are male.\n'
                             'Their passenger class is 3.\n'
                             'Their age is 22.0.\n'
                             'They paid $7.125 for the trip.'},
                   {'from': 'gpt', 'value': '0'},
                   {'from': 'human',
                    'value': 'The passenger embarked from C.\n'
                             'They are female.\n'
                             'They have 1 siblings and spouses.\n'
                             'Their passenger class is 2.\n'
                             'Their age is 27.0.\n'
                             'They paid $13.8583 for the trip.'},
                   {'from': 'gpt', 'value': '1'},
                   {'from': 'human',
                    'value': 'The passenger embarked from S.\n'
                             'They are male.\n'
                            

{'conversations': [{'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 3.\nTheir age is 22.0.\nThey paid $7.125 for the trip.',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'The passenger embarked from C.\nThey are female.\nThey have 1 siblings and spouses.\nTheir passenger class is 2.\nTheir age is 27.0.\nThey paid $13.8583 for the trip.',
   'role': 'user'},
  {'content': '1', 'role': 'assistant'},
  {'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 1.\nTheir age is 61.0.\nThey paid $33.5 for the trip.',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 2.\nTheir age is 19.0.\nThey paid $10.5 for the trip.',
   'role': 'user'},
  {'content': '1', 'role': 'assistant'},
  {'content': 'The passenger embarked from S.\nThey are male.\nTheir passenger class is 3.\nThey paid $9.5 for the trip.',

# Fine‑Tuning Setup 🧠

We define a simple chat template and configure the SFT (Supervised Fine Tuning) trainer. The template tells the model how to format the input and output during training.

In [8]:
chat_template = """Below describes some details about some passengers who went on the Titanic.
Predict whether they survived or perished based on their characteristics.
Output 1 if they survived, and 0 if they died.
>>> Passenger Details:
{INPUT}
>>> Did they survive?
{OUTPUT}"""

from unsloth import apply_chat_template
train_dataset = apply_chat_template(
    train_dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/712 [00:00<?, ? examples/s]

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",   # safer across versions
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/712 [00:00<?, ? examples/s]

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 712 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.616100
2,1.563200
3,1.571300
4,1.527000
5,1.392900
6,1.232000
7,1.061500
8,1.005800
9,0.910300
10,0.816400


# Quick Inference ⚡

After training, we run the model on a sample passenger to see its prediction. We prepare the input in chat format and generate the model’s response.

In [12]:
# Prepare a sample query and generate a prediction

# Switch the model to inference mode
FastLanguageModel.for_inference(model)

# Define a test passenger description
messages = [                    # Change below!
    {"role": "user", "content": 'The passenger embarked from S.\n'\
                                'They are male.\n'\
                                'They have 1 siblings and spouses.\n'\
                                'Their passenger class is 3.\n'\
                                'Their age is 22.0.\n'\
                                'They paid $7.25 for the trip.'},
]
device = "cuda" if torch.cuda.is_available() else "cpu"
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to(device)

# Set up a streamer to display the model's output as it is generated
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Generate the prediction
outputs = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id,
    use_cache=False,
    attention_mask=torch.ones_like(input_ids)
)

0<|end_of_text|>


# Evaluate on Test Data 📊

Now let's evaluate the model on the held‑out test set. We'll format the test data just like the training data, run predictions, and compute accuracy and other metrics.

In [13]:
# Convert test dataset to the same format as training data
test_dataset_formatted = to_sharegpt(
    test_dataset,
    merged_prompt = \
        "[[The passenger embarked from {Embarked}.]]"\
        "[[\nThey are {Sex}.]]"\
        "[[\nThey have {Parch} parents and childen.]]"\
        "[[\nThey have {SibSp} siblings and spouses.]]"\
        "[[\nTheir passenger class is {Pclass}.]]"\
        "[[\nTheir age is {Age}.]]"\
        "[[\nThey paid ${Fare} for the trip.]]",
    conversation_extension = 1,  # Keep it 1 for individual predictions
    output_column_name = "Survived",
)


In [14]:
# Process each test sample
predictions = []
actual_labels = []

device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
# Helper function to extract prediction from model output
def extract_prediction(response):
    """Extract 0 or 1 from model response"""
    response = response.strip().lower()
    if '1' in response:
        return 1
    elif '0' in response:
        return 0
    else:
        # Handle edge cases where model doesn't give clear 0/1
        if 'survived' in response or 'yes' in response:
            return 1
        else:
            return 0

In [16]:
for i, sample in enumerate(test_dataset_formatted):
    # Extract the input content (without the expected output)
    user_content = sample['conversations'][0]['value']  # The formatted passenger details

    messages = [
        {"role": "user", "content": user_content}
    ]

    # Tokenize
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(device)

    # Generate prediction
    with torch.no_grad():
      outputs = model.generate(
            input_ids,
            streamer=text_streamer,
            max_new_tokens=10,  # Short response needed (just 0 or 1)
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False,  # Use greedy decoding for consistent results
            temperature=0.1,
            attention_mask=torch.ones_like(input_ids)  # Explicit attention mask
        )
    # Decode the response
    response = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

    # Extract prediction (0 or 1)
    predicted_label = extract_prediction(response)
    actual_label = sample['conversations'][1]['value']  # The actual survival status

    predictions.append(predicted_label)
    actual_labels.append(int(actual_label))

    # print(f"Sample {i+1}: Predicted={predicted_label}, Actual={actual_label}")

0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
1<|end_of_text|>
0<|end_of_text|>
0<|end_of_text|>
0<|end_of_text

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Simple text-based confusion matrix with percentages
def print_detailed_confusion_matrix(actual, predicted):
    cm = confusion_matrix(actual, predicted)
    tn, fp, fn, tp = cm.ravel()
    total = len(actual)

    print("\nDetailed Confusion Matrix:")
    print("="*40)
    print(f"{'':12} {'Predicted':>20}")
    print(f"{'':12} {'Died':>8} {'Survived':>8} {'Total':>8}")
    print("-"*40)
    print(f"{'Died':12} {tn:5d} ({tn/total*100:4.1f}%) {fp:5d} ({fp/total*100:4.1f}%) {tn+fp:5d}")
    print(f"{'Survived':12} {fn:5d} ({fn/total*100:4.1f}%) {tp:5d} ({tp/total*100:4.1f}%) {fn+tp:5d}")
    print("-"*40)
    print(f"{'Total':12} {tn+fn:5d} {fp+tp:5d} {total:5d}")

    print(f"\nModel Performance:")
    print(f"Correctly predicted deaths: {tn}/{tn+fp} = {tn/(tn+fp)*100:.1f}%")
    print(f"Correctly predicted survivals: {tp}/{fn+tp} = {tp/(fn+tp)*100:.1f}%")
    print(f"Overall accuracy: {(tn+tp)/total*100:.1f}%")

# Call the function
print_detailed_confusion_matrix(actual_labels, predictions)


Detailed Confusion Matrix:
                        Predicted
                 Died Survived    Total
----------------------------------------
Died            98 (54.7%)     1 ( 0.6%)    99
Survived        64 (35.8%)    16 ( 8.9%)    80
----------------------------------------
Total          162    17   179

Model Performance:
Correctly predicted deaths: 98/99 = 99.0%
Correctly predicted survivals: 16/80 = 20.0%
Overall accuracy: 63.7%


In [18]:
# Decode the response
response = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

# Extract prediction (0 or 1)
predicted_label = extract_prediction(response)
actual_label = sample['conversations'][1]['value']  # The actual survival status

predictions.append(predicted_label)
actual_labels.append(int(actual_label))

print(f"Sample {i+1}: Predicted={predicted_label}, Actual={actual_label}")

Sample 179: Predicted=0, Actual=1


# Conclusion ✅

Great job! You loaded data, prepared it, fine‑tuned a model, and evaluated its performance. Keep experimenting and improving your model! 🚀